# Learning how to use PyDDM

In [2]:
#@title Install PyDDM on Google Colab
!pip -q install git+https://github.com/mwshinn/PyDDM

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Import packages

In [3]:
import pyddm
import pyddm.plot
import matplotlib.pyplot as plt
import pandas
import numpy as np

# define collapsing bound w/ leak, plot via GUI

In [4]:
model = pyddm.gddm(drift=lambda x,leak,driftrate : driftrate - x*leak,
                   noise=1,
                   bound=lambda t,initial_B,collapse_rate : initial_B * np.exp(-collapse_rate*t),
                   starting_position="x0",
                   parameters={"leak": (0, 2),
                               "driftrate": (-3, 3),
                               "initial_B": (.5, 1.5),
                               "collapse_rate": (0, 10),
                               "x0": (-.9, .9)})

pyddm.plot.model_gui_jupyter(model)
# pyddm.plot.model_gui(model) # If not using a Jupyter notebook or Google Colab

Output()

In [ ]:
# tweak the core model / GUI myself

In [ ]:
model = pyddm.gddm(
    drift='drift',
    noise=1,
    bound=lambda t,initial_B,collapse_rate : initial_B * np.exp(-collapse_rate*t),
    starting_position="x0",
    T_dur=0.5,
    parameters={
        "drift": (0,1),
        "initial_B": (.5, 1.5),
        "collapse_rate": (0, 10),
        "x0": (-.9, .9)})

pyddm.plot.model_gui_jupyter(model)

Output()

# simple model recovery analysis

In [ ]:
# specify generating model
true_model = pyddm.gddm(drift=0.5, noise=1.0, bound=1, starting_position=0.3, nondecision=0.2)

# obtain its solution
true_solution = true_model.solve()

# specify fitting model
fitting_model = pyddm.gddm(drift="d", noise=1.0, bound="B", starting_position="x0", nondecision=0.2, parameters={"d": (-2,2), "B": (0.3, 2), "x0": (-.8, .8)}, name='dummy')

# generate data by sampling from the solution of the generating model
data = true_solution.sample(100000)

# apply the fitting model to simulated data
fitting_model.fit(data, lossfunction=pyddm.LossBIC, verbose=False)

# get printout of results
fitting_model.show()

# get fixed & fitted parameter values
fitting_model.parameters()

# visualize fitting success
pyddm.plot.plot_fit_diagnostics(model=fitting_model, sample=data)

AttributeError: module 'pyddm' has no attribute 'fit'

In [ ]:
model = pyddm.gddm(drift = 0.5, noise = 0.1, bound = 1, starting_position=0, nondecision=0.2, mixture_coef=0, T_dur=4)

sol = model.solve()


sol.mean_decision_time()
sol.prob('correct')
sol.prob('error')
sol.prob_forced('correct')
sol.prob_forced('error')
#sol.pdf('correct')
sol.pdf_undec()


plt.plot(sol.t_domain, sol.pdf('correct'))
plt.plot(sol.t_domain, sol.pdf('error'))
plt.plot(sol.t_domain, sol.pdf_undec())
plt.xlabel('RT')
plt.ylabel('cumulative probability')
plt.legend(['correct RTs', 'incorrect RTs', 'undecided'])
plt.show()

model.simulate_trial()

# try time-varying drift simulation (arousal vector example)

In [ ]:
# first generate 3 trials of data, each of which has a different number of arousal recordings
df = pandas.DataFrame({"choice_correct": [1, 0, 1], "arousal": [(.5, .7, .8, .9), (.2, .3, .4, .1, .1), (1.1, 1.3)], "RT": [.65, .94, .30]})

# then sample from that dataframe
sim = pyddm.Sample.from_pandas_dataframe(df, rt_column_name='RT', choice_column_name='choice_correct')

# define function that determines moment-by-moment drift
def find_drift(t, arousal_vec):
  t_bin = int(t // .2) # bins time into 200ms bins
  t_bin = min(len(arousal_vec)-1, t_bin) # ensures we don't go past the final arousal bin
  return arousal_vec[t_bin]

# define model
m = pyddm.gddm(drift=lambda t,arousal_scale,arousal : find_drift(t, arousal)*arousal_scale,
               parameters={'arousal_scale': (0,2)},
               conditions=['arousal'])

# plot
pyddm.plot.model_gui_jupyter(m, sim, data_dt=.2)
sim.mean_decision_time()

Info: Simulating trial 0
Info:pyddm:Simulating trial 0


AttributeError: 'set' object has no attribute 'keys'

# try changing signal strengths example


In [ ]:
def coherence_changes(t, coh1, coh2, coh3, drift_multiplier, t1, t2):
    if t<t1:
        return coh1*drift_multiplier
    elif t<t1+t2:
        return coh2*drift_multiplier
    else:
        return coh3*drift_multiplier

m = pyddm.gddm(drift=coherence_changes,
               parameters={"drift_multiplier": (-3,3)},
               conditions=["coh1", "coh2", "coh3", "t1", "t2"])
pyddm.plot.model_gui_jupyter(m, conditions={"coh1": [0, .5, 1], "coh2": [0, .5, 1], "coh3": [0, .5, 1], "t1": [.2, .4, .6], "t2": [.2, .4, .6]})


m.show()

Output()

Model drift_multiplier information:
Choices: 'correct' (upper boundary), 'error' (lower boundary)
Drift component DriftEasy:
    easy_drift
    Fitted parameters:
    - drift_multiplier: 2.451291
Noise component NoiseConstant:
    constant
    Fixed parameters:
    - noise: 1.000000
Bound component BoundConstant:
    constant
    Fixed parameters:
    - B: 1.000000
IC component ICPointRatio:
    An arbitrary starting point expressed as a proportion of the distance between the bounds.
    Fixed parameters:
    - x0: 0.000000
Overlay component OverlayChain:
    Overlay component OverlayNonDecision:
        Add a non-decision by shifting the histogram
        Fixed parameters:
        - nondectime: 0.000000
    Overlay component OverlayUniformMixture:
        Uniform distribution mixture model
        Fixed parameters:
        - umixturecoef: 0.020000



# adjust example to be more like our experiment

## specification 1: cue & thinning as conditions

In [ ]:
def coherence_changes(t, coherence, cue, thinning, signal1_onset, noise2_onset, signal2_onset):
    if t<signal1_onset:
      return cue/thinning
    elif t>signal1_onset and t<noise2_onset:
      return coherence + (cue/thinning)
    elif t>noise2_onset and t<signal2_onset:
      return cue/thinning
    else:
      return coherence + (cue/thinning)

m = pyddm.gddm(drift=coherence_changes,
               mixture_coef=0,
               T_dur=3.5,
               parameters={"signal1_onset": (0.64, 1.26), "noise2_onset": (1, 2.3), "signal2_onset": (1.73, 3.26), "coherence": 0.7},
               conditions=["cue", "thinning"])
m.show()

pyddm.plot.model_gui_jupyter(m, conditions={"cue": [0.2, 0.5, 0.8], "thinning": [4, 8, 12]})

## specification 2: all variables as parameters, more intuitive definitions of epoch durations

In [ ]:
def coherence_changes(t, coherence, cue, thinning, noise1_duration, signal1_duration, noise2_duration):
    if t < noise1_duration:
      return cue/thinning
    elif t > noise1_duration and t < noise1_duration + signal1_duration:
      return coherence + (cue/thinning)
    elif t > noise1_duration + signal1_duration and t < noise1_duration + signal1_duration + noise2_duration:
      return cue/thinning
    else:
      return coherence + (cue/thinning)

m = pyddm.gddm(drift=coherence_changes,
               mixture_coef=0,
               T_dur=3.5,
               bound=1.5,
               nondecision=0,
               parameters={"noise1_duration": (0, 1.25), "signal1_duration": (0.42, 1), "noise2_duration": (0.63, 1.25),
                           "coherence": (0.5,1), "cue": (0,1), "thinning": (4,36)})
m.show()

pyddm.plot.model_gui_jupyter(m)

m.simulate_trial()

Model thinning information:
Choices: 'correct' (upper boundary), 'error' (lower boundary)
Drift component DriftEasy:
    easy_drift
    Fitted parameters:
    - coherence: 0.599125
    - cue: 0.421772
    - thinning: 21.010334
    - noise1_duration: 0.929617
    - signal1_duration: 0.791329
    - noise2_duration: 0.769931
Noise component NoiseConstant:
    constant
    Fixed parameters:
    - noise: 1.000000
Bound component BoundConstant:
    constant
    Fixed parameters:
    - B: 1.500000
IC component ICPointRatio:
    An arbitrary starting point expressed as a proportion of the distance between the bounds.
    Fixed parameters:
    - x0: 0.000000
Overlay component OverlayChain:
    Overlay component OverlayNonDecision:
        Add a non-decision by shifting the histogram
        Fixed parameters:
        - nondectime: 0.000000
    Overlay component OverlayUniformMixture:
        Uniform distribution mixture model
        Fixed parameters:
        - umixturecoef: 0.000000



## specification 3: object-oriented so that i can visualize the DV on single trials

In [ ]:
# go object-oriented to try simulating the trial
from pyddm import Model
from pyddm.models import DriftConstant, NoiseConstant, BoundConstant, OverlayNonDecision, ICPointSourceCenter
from pyddm.functions import display_model

model = Model(name='Simple model',
              drift=DriftConstant(drift=2.2),
              noise=NoiseConstant(noise=1.5),
              bound=BoundConstant(B=1.1),
              overlay=OverlayNonDecision(nondectime=.1),
              dx=.001, dt=.01, T_dur=2)
display_model(model)

model.simulate_trial()

array([-1.21236354e-13, -1.21236354e-13, -1.21236354e-13, -1.21236354e-13,
       -1.21236354e-13, -1.21236354e-13, -1.21236354e-13, -1.21236354e-13,
       -1.21236354e-13, -1.21236354e-13, -1.21236354e-13,  1.33238761e-01,
        3.88175819e-01,  6.99265892e-02,  2.91958396e-01,  1.87549788e-01,
        5.05038456e-01,  7.16956235e-01,  6.63074754e-01,  1.06685487e+00,
        1.25097666e+00])